# Symbolic developments
One of the major advantages of the kernuller package is that the combiner matrices are managed in a symbolic format.

While it is not case for default propagation when using `kernuller.get_I()` method, this symbolic format enables to manipulate the symbolic representation, not only of the combiner matrix, but also of the propagated light.

Depending on the goal, one can develop the equations corresponding to the requested type of propagation including for axample different types of aberrations, or the location of the apertures.

In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
#import kernuller_class as kernuller
import kernuller
import astropy.coordinates
import astropy.units as u

from time import time

# =========================================================================
# =========================================================================
def mas2rad(x):
    ''' Convenient little function to convert milliarcsec to radians '''
    return(x * 4.8481368110953599e-09) # = x*np.pi/(180*3600*1000)

# =========================================================================
# =========================================================================
def rad2mas(x):
    '''  convert radians to mas'''
    return(x / 4.8481368110953599e-09) # = x / (np.pi/(180*3600*1000))

# =========================================================================
# =========================================================================

## The first step is to build a model

In [ ]:
mymatrix = np.load("../data/4T_matrix.npy", allow_pickle=True)
mykernuller = kernuller.kernuller(kernuller.VLTI,5.e-6)
mykernuller.build_model_from_matrix(mymatrix)
mykernuller.Ks = sp.Matrix(kernuller.pairwise_kernel(6, blankcols=1))

In [ ]:
Mn = np.array(sp.N(mykernuller.Ms), dtype=np.complex64)
Mn2 = np.vstack((Mn[:1,:], np.zeros_like(Mn[1,:]), Mn[1:,:]))
fig, axs = mykernuller.plot_outputs_smart(Mn2, nx=2,legendoffset=(1.5,0.5), dpi=100,
                                          plotsize=3, osfrac=0.1, title=False, mainlinewidth=0.03,
                                          labelsize=10,legendstring="center left", outputontop=True,
                                          labels=False, onlyoneticklabel=False)

## Building the relevant equation
### Position of the apertures

In [ ]:
X = sp.Matrix(sp.symbols('X:{}'.format(mykernuller.Na+1), real=True))
Y = sp.Matrix(sp.symbols('Y:{}'.format(mykernuller.Na+1), real=True))

## Complex amplitude of light sampled by this array
We need $\lambda$ (wavelength), $\alpha$ and $\beta$ (sky position relative to optical axis) $\zeta$ (an amplitude term for each input) $\gamma$ (an input phase error term we didn't use here).

In [ ]:
lamb = sp.symbols("lambda", real=True)
alpha = sp.symbols("alpha", real=True)
beta = sp.symbols("beta", real=True)
zeta = sp.Matrix(sp.symbols('zeta:{}'.format(mykernuller.Na), real=True))
gamma = sp.Matrix(sp.symbols('gamma:{}'.format(mykernuller.Na), real=True))

In [ ]:
zeta

For now they are just symbols, but now we can use it in an equation.

In [ ]:
z = sp.Matrix([zeta[i]*sp.exp(sp.I*(2*sp.pi/lamb)*(alpha*X[i] + beta*Y[i])) for i in range(mykernuller.Na)])
z

## The equation for the output electric field
Very easy, you just have to multiply it by the combiner matrix

In [ ]:
anE = mykernuller.Ms@z
anE

One might prefer it under a different form:

## Preparing substitutions
Now we want to replace the constants by the value they will take. This can be prepared in a list. For now let us only replace $\zeta$ and look at the result:

In [ ]:
thesubs = []
for i in range(mykernuller.Na):
    #thesubs.append((X[i],X[i]-X[0]))
    #thesubs.append((Y[i],Y[i]-Y[0]))
    thesubs.append((zeta[i], 1))
#thesubs.append((X[0],0))
#thesubs.append((Y[0],0))

In [ ]:
anE2 = sp.expand_mul(anE.subs(thesubs))
anE2

## Computing the output intensities

In [ ]:
anI =  sp.matrix_multiply_elementwise(sp.conjugate(anE2), anE2)
anI = sp.expand_mul(anI)
anI

## Now let us look at the kernel vector

In [ ]:
kappa = mykernuller.Ks@anI
kappa

## Finer examination of a single kernel
The sympy method `expand_complex()` is helpful to show the trigonometric form of the output (We knew this had to be real!)

In [ ]:
aker = anI[1]-anI[2]
sinform = sp.expand_complex(aker)
sinform

## Perspectives:
From there, it becomes really easy to extract
* derivatives of those expressions `sp.diff(expr, x)`
* taylor series expansions `sp.series(expr,x, x0, n)`
And more!

From there, one could for example optimize the position of an aperture to maximize the kernel signal on a target.

# A form that is usabale numerically
## The lambdify method
As an example, let us create a kernel response map

First, we must substitute the parameters that are going to remain constant, constituting the expression `sinapplied`

In [ ]:
thesubs = []
for i in range(mykernuller.Na):
    #thesubs.append((X[i],X[i]-X[0]))
    #thesubs.append((Y[i],Y[i]-Y[0]))
    thesubs.append((X[i], mykernuller.pups[i, 0]))
    thesubs.append((Y[i], mykernuller.pups[i, 1]))
    thesubs.append((zeta[i], 1))
thesubs.append((lamb, 3.6e-6))
sinapplied = sinform.subs(thesubs)
sinapplied

Then we must turn this expression into a `numpy` function that can be executed very fast.

In [ ]:
mykerout = sp.utilities.lambdify((alpha, beta), sp.expand_complex(kappa).subs(thesubs))

Now `mykerout` is a numpy function that returns a kernel vector from the position of a source.

* It is *fast*
* It is vectorized

In [ ]:
%timeit akernel = mykerout(1e-6, 2e-6)

In [ ]:
xx, yy = np.meshgrid(np.linspace(-15,15,1024), np.linspace(-15,15,1024))
xxr = mas2rad(xx)
yyr = mas2rad(yy)
# Here we check how long it takes to compute the map
start = time()
amap = mykerout(xxr,yyr)
amap = np.squeeze(amap) # Removing an unwanted dimension
print("Map computed in %.2f seconds"%(time() - start))

In [ ]:
fig, axs = mykernuller.plot_response_maps(amap, title=False,cbar_label="Kernel-null value (single aperture flux)",
                                          extent=[np.min(xx),np.max(xx), np.min(yy), np.max(yy)],
                                         plotsize=4, dpi=100)

In [ ]:
from lmfit import minimize, Parameters
from tqdm import tqdm
from xara import mas2rad

## Build a function that returns model signal
Just a little bit of packaging, unit conversion...

In [ ]:
def get_kn_signal(params):
    alpha = mas2rad(params["alpha"])
    beta = mas2rad(params["beta"])
    ic = params["ic"]
    kn_sig = ic * mykerout(alpha, beta).flatten()
    return kn_sig

## Build a function that returns the residual

In [ ]:
def get_kn_residual(params, y):
    return y -  get_kn_signal(params)

## Build a parameter object

In [ ]:
params = Parameters()
params.add("alpha", value=5, min=-10, max=10)
params.add("beta", value=8, min=-10, max=10)
params.add("ic",value=3, min=0, max=20)

## A Monte Carlo simulation for noisy data
The measured data will be represented by `ys`

Here, I only simulate read noise on the measured intensities

In [ ]:
noisevec = np.random.normal(scale=0.1, size=(10000,3))
ys = get_kn_signal(params)
noisy = noisevec + ys.T

Let us change the starting point slightly, so that it is not too easy

In [ ]:
params = Parameters()
params.add("alpha", value=4.5, min=-10, max=10)
params.add("beta", value=7, min=-10, max=10)
params.add("ic",value=2.5, min=0, max=20)

In [ ]:
soluce = minimize(get_kn_residual, params, args=(ys,))
soluce

## Now to do the model-fit for each of the realizations

In [ ]:
sols = []
alphas = []
betas = []
ics = []
res = []
for y in tqdm(noisy):
    soluce = minimize(get_kn_residual, params, args=(y,))
    sols.append(soluce.params)
    alphas.append(soluce.params["alpha"].value)
    betas.append(soluce.params["beta"].value)
    ics.append(soluce.params["ic"].value)
    res.append(get_kn_residual(params, y))
alphas = np.array(alphas)
betas = np.array(betas)
ics = np.array(ics)
plt.figure(dpi=200)
plt.hist2d(alphas, betas, bins=50)
plt.xlabel(r"$\alpha$ (mas)")
plt.ylabel(r"$\beta$ (mas)")
plt.gca().set_aspect("equal")
plt.title("A distribution of the fitted position")
plt.show()
get_kn_residual(params, y)

## For a more standard way of looking at that kind of data, use corner plots

In [ ]:
import corner
figure = corner.corner(np.vstack((alphas, betas, ics)).T,labels=[r"$\alpha$ (mas)", r"$\beta$ (mas)", "intensity"])

## Can also express the result in separation and position angle

In [ ]:
rhos = np.sqrt(alphas**2+betas**2)
cpform = alphas + 1j * betas
rhos = np.abs(cpform)
thetas = (np.angle(cpform)+np.pi/2)*180/np.pi

In [ ]:
figure = corner.corner(np.vstack((rhos, thetas, ics)).T,labels=[r"$\rho$ (mas)", r"$\theta$ (deg)", "intensity"])